In [2]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import math
class SpeedDataset(Dataset):
    def __init__(self, directory, sequence_length=5):
        self.data = []
        self.sequence_length = sequence_length
        self.max_speed = 0  # Variable to store the maximum speed found
        self.preprocess_data(directory)
        # Calculate number of classes based on the range from 30 to 105
        self.num_classes = (105 - 30) // 10 +1   # From class 30-39, ..., 100-105

    def preprocess_data(self, directory):
        for filename in os.listdir(directory):
            if filename.endswith('.txt'):
                speed = float(filename.split('_')[-1].replace('.txt', ''))
                # Update maximum speed if current speed is greater
                if speed > self.max_speed:
                    self.max_speed = speed
                # Convert speed to class, starting range from 30
                if speed < 30:
                    speed_class = 0  # Handle speeds below 30 as class 0
                else:
                    speed_class = (int(speed) - 30) // 10
                filepath = os.path.join(directory, filename)
                with open(filepath, 'r') as file:
                    track_data = {}
                    for line in file:
                        frame, track_id, x1, y1, x2, y2 = map(float, line.strip().split(',')[:6])
                        if track_id not in track_data:
                            track_data[track_id] = []
                        track_data[track_id].append([x1,y1,x2,y2,(x1+x2)/2, (y1+y2)/2])

                    for track_id, frames in track_data.items():
                        if len(frames) >= self.sequence_length:
                            features = []
                            for i in range(1, len(frames)):
                                current_frame = frames[i]
                                previous_frame = frames[i-1]
                                data_d = []
                                distance = 0
                                distance1 = 0
                                distance2 = 0
                                for i in range(4):
                                    data_d.append(abs(current_frame[i] - previous_frame[i]))
                                for j in range(4,6):
                                    distance = ((current_frame[j] - previous_frame[j])**2) + distance
                                for j in range(0,2):
                                    distance1 = ((current_frame[j] - previous_frame[j])**2) + distance1
                                for j in range(2,4):
                                    distance2 = ((current_frame[j] - previous_frame[j])**2) + distance2
                                data_d.append(math.sqrt(distance))
                                data_d.append(math.sqrt(distance1))
                                data_d.append(math.sqrt(distance2))
                                features.append(data_d)
                            for i in range(len(features) - self.sequence_length + 1):
                                sequence = features[i:i + self.sequence_length]
                                self.data.append((sequence, speed_class))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputs, output = self.data[idx]
        return torch.tensor(inputs, dtype=torch.float32), torch.tensor(output, dtype=torch.long)


In [9]:
import torch
import torch.nn as nn

class SpeedPredictor(nn.Module):
    def __init__(self, sequence_length, feature_size, embedding_dim, hidden_dim, output_size):
        super(SpeedPredictor, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=feature_size, out_channels=embedding_dim, kernel_size=1)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, num_layers = 3)
        self.fc2 = nn.Linear(64, output_size)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(in_channels=hidden_dim, out_channels=64, kernel_size=1)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.conv1(x)
        x = x.permute(0, 2, 1)
        x, _ = self.lstm(x)
        x = x[:, -1, :]  # Use only the last output of LSTM for further processing
        x = x.unsqueeze(2)  # Adjust dimensions for the second conv layer
        x = self.conv2(x)
        x = x.squeeze(2)  # Flatten the output for FC layers
        return self.relu(self.fc2(x))


In [10]:
import torch

# Check for CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [11]:

def train(model, train_loader, test_loader, criterion, optimizer, scheduler, epochs):
    for epoch in range(epochs):
        model.train()
        Train_total_loss = 0
        correct_train = 0
        total_train = 0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Ensure inputs and targets are on the same device as model
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()  # First, update the parameters with the current learning rate
            
            Train_total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            correct_train += (predicted == targets).sum().item()
            total_train += targets.size(0)
        
        scheduler.step()  # After optimizer updates, adjust the learning rate

        if epoch % 10 == 0 or epoch == epochs - 1:
            model.eval()
            correct_test = 0
            total_test = 0
            total_loss = 0
            with torch.no_grad():
                for inputs, targets in test_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    total_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    correct_test += (predicted == targets).sum().item()
                    total_test += targets.size(0)
                    
            train_accuracy = 100 * correct_train / total_train
            test_accuracy = 100 * correct_test / total_test
            print(f'Epoch {epoch+1}: Train Loss: {Train_total_loss / len(train_loader)} Test Loss: {total_loss / len(train_loader)}, '
                  f'Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%')


In [12]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.optim import SGD
from torch.optim.lr_scheduler import ExponentialLR

# Assuming your SpeedPredictor and SpeedDataset are already defined and imported

# Initialize datasets
train_dataset = SpeedDataset('outputs', sequence_length=25)
test_size = int(0.15 * len(train_dataset))  # 10% for testing
train_size = len(train_dataset) - test_size  # 90% for training

# Split the dataset
train_subset, test_subset = random_split(train_dataset, [train_size, test_size])

# Data loaders
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=10, shuffle=False)

# Model setup
max_classes = train_dataset.num_classes
model = SpeedPredictor(sequence_length=25, feature_size=7, embedding_dim=64, hidden_dim=128, output_size=max_classes)
model.to(device)  # Ensure model is on the appropriate device

# Loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device)  # Ensure loss function is on the appropriate device
optimizer = SGD(model.parameters(), lr=0.08)
scheduler = ExponentialLR(optimizer, gamma=0.99)  # Learning rate scheduler

# Train the model
train(model, train_loader, test_loader, criterion, optimizer, scheduler, epochs=200)


Epoch 1: Train Loss: 2.070943194277146 Test Loss: 1.17000785353137, Train Accuracy: 15.95%, Test Accuracy: 16.29%
Epoch 11: Train Loss: 2.058769822412846 Test Loss: 1.1643859992424648, Train Accuracy: 17.81%, Test Accuracy: 17.46%
Epoch 21: Train Loss: 2.0513469223298277 Test Loss: 1.1558385435857026, Train Accuracy: 18.24%, Test Accuracy: 18.25%
Epoch 31: Train Loss: 2.028886291326261 Test Loss: 1.1464399245439791, Train Accuracy: 21.66%, Test Accuracy: 25.15%
Epoch 41: Train Loss: 1.9783088106150721 Test Loss: 1.1289796563340169, Train Accuracy: 25.99%, Test Accuracy: 25.33%
Epoch 51: Train Loss: 1.8723801827313853 Test Loss: 1.048267317753212, Train Accuracy: 32.17%, Test Accuracy: 33.41%
Epoch 61: Train Loss: 1.6890615145949757 Test Loss: 0.9478972773341572, Train Accuracy: 42.04%, Test Accuracy: 42.75%
Epoch 71: Train Loss: 1.4216347925511061 Test Loss: 0.7820450428361986, Train Accuracy: 54.26%, Test Accuracy: 54.74%
Epoch 81: Train Loss: 1.173432003487559 Test Loss: 0.6788297959

In [15]:
def print_confusion_matrix_and_report(all_targets, all_preds):
    print(confusion_matrix(all_targets, all_preds))
    print(classification_report(all_targets, all_preds, target_names=[f'Class {30 + i * 10}-{39 + i * 10}' for i in range(max(all_targets) + 1)]))


In [17]:
def evaluate(model, test_loader):
    model.eval()
    correct_test = 0
    total_test = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Ensure inputs and targets are on the same device as model
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            correct_test += (predicted == targets).sum().item()
            total_test += targets.size(0)
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    test_accuracy = 100 * correct_test / total_test
    print("Confusion Matrix and Classification Report:")
    print_confusion_matrix_and_report(all_targets, all_preds)
    return test_accuracy
evaluate(model, test_loader)  

Confusion Matrix and Classification Report:
[[447   0   0   0   0   0   0   0]
 [  1 424   0   0   0   0   0   0]
 [  0   0 375   0   0   0   0   0]
 [  1   0   0 356   2   0   0   0]
 [  0   0   0   0 293   1   0   0]
 [  0   0   0   0   0 140   0   0]
 [  0   0   0   0   0   0 160   0]
 [  0   0   1   0   0   0   0 101]]
               precision    recall  f1-score   support

  Class 30-39       1.00      1.00      1.00       447
  Class 40-49       1.00      1.00      1.00       425
  Class 50-59       1.00      1.00      1.00       375
  Class 60-69       1.00      0.99      1.00       359
  Class 70-79       0.99      1.00      0.99       294
  Class 80-89       0.99      1.00      1.00       140
  Class 90-99       1.00      1.00      1.00       160
Class 100-109       1.00      0.99      1.00       102

     accuracy                           1.00      2302
    macro avg       1.00      1.00      1.00      2302
 weighted avg       1.00      1.00      1.00      2302



99.73935708079931